In [ ]:
from __future__ import print_function
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
#from keras.utils import np_utils
import pickle
from keras.layers import LSTM
from keras.layers import BatchNormalization

from sklearn.model_selection import StratifiedKFold


In [ ]:
!pip install np_utils

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma


In [ ]:


def show_confusion_matrix(validations, predictions):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap="Spectral",
                linecolor='white',
                linewidths=1,
                xticklabels=["Rest", "Stress"],
                yticklabels=["Rest", "Stress"],
                annot=True,
                fmt="d")
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()



In [ ]:

def show_basic_dataframe_info(dataframe,
                              preview_rows=20):

    """
    This function shows basic information for the given dataframe
    Args:
        dataframe: A Pandas DataFrame expected to contain data
        preview_rows: An integer value of how many rows to preview
    Returns:
        Nothing
    """

    # Shape and how many rows and columns
    print("Number of columns in the dataframe: %i" % (dataframe.shape[1]))
    print("Number of rows in the dataframe: %i\n" % (dataframe.shape[0]))
    print("First 20 rows of the dataframe:\n")
    # Show first 20 rows
    print(dataframe.head(preview_rows))
    print("\nDescription of dataframe:\n")
    # Describe dataset like mean, min, max, etc.
    # print(dataframe.describe())



In [ ]:

def read_data(file_path):

    """
    This function reads the accelerometer data from a file
    Args:
        file_path: URL pointing to the CSV file
    Returns:
        A pandas dataframe
    """


    column_names = [
                    'user-id',
                    'timestamp',
                    'activity',
                    'HR',
                    'HRV',
                    'Raw',
                    'EDA',
                   ]
    df = pd.read_csv(file_path,
                     header=None,
                     names=column_names)
    # Last column has a ";" character which must be removed ...
   # df['z-axis'].replace(regex=True,
   #   inplace=True,
   #   to_replace=r';',
   #   value=r'')
    # ... and then this column must be transformed to float explicitly
    #df['z-axis'] = df['z-axis'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df.dropna(axis=0, how='any', inplace=True)

    return df



In [ ]:

def convert_to_float(x):

    try:
        return np.float(x)
    except:
        return np.nan



In [ ]:

# Not used right now
def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma



In [ ]:

def plot_axis(ax, x, y, title):

    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)



In [ ]:

def plot_activity(activity, data):

    fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=4,
         figsize=(15, 10),
         sharex=True)
    plot_axis(ax0, data['timestamp'], data['HR'], 'HR')
    plot_axis(ax1, data['timestamp'], data['HRV'], 'HRV')
    plot_axis(ax2, data['timestamp'], data['Raw'], 'Raw')
    plot_axis(ax3, data['timestamp'], data['EDA'], 'EDA')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()


In [ ]:


def create_segments_and_labels(df, time_steps, step, label_name):

    """
    This function receives a dataframe and returns the reshaped segments
    of x,y,z acceleration as well as the corresponding labels
    Args:
        df: Dataframe in the expected format
        time_steps: Integer value of the length of a segment that is created
    Returns:
        reshaped_segments
        labels:
    """

    # x, y, z acceleration as features
    N_FEATURES = 4
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['HR'].values[i: i + time_steps]
        ys = df['HRV'].values[i: i + time_steps]
        zs = df['Raw'].values[i: i + time_steps]
        zs1 = df['EDA'].values[i: i + time_steps]
        label = stats.mode(df[label_name][i: i + time_steps])[0]
        segments.append([xs, ys, zs, zs1])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels



In [ ]:
from keras.utils import to_categorical

In [ ]:
!pip install np_utils

In [ ]:


# ------- THE PROGRAM TO LOAD DATA AND TRAIN THE MODEL -------

# Set some standard parameters upfront
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
print('keras version ', keras.__version__)

LABELS = ["rest",
          "medium stress",
          "high stress",
          "low stress",
          "pre_anxiety",
          "anxiety_attack",]
# The number of steps within one time segment
TIME_PERIODS = 300 #was 80
# The steps to take from one segment to the next; if this value is equal to
#TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 300 #was 40

# %%

print("\n--- Load, inspect and transform data ---\n")

# Load data set containing all the data from csv
#df = read_data('Data/WISDM_ar_v1.1_raw.txt')
df = read_data('final_dataset.csv')

df1 = read_data('final_dataset.csv')


# Describe the data
show_basic_dataframe_info(df, 20)

#df['activity'].value_counts().plot(kind='bar',
#                                   title='Training Examples by Activity Type')
#plt.show()

#df['user-id'].value_counts().plot(kind='bar',
#                                  title='Training Examples by User')
#plt.show()

#for activity in np.unique(df["activity"]):
#    subset = df[df["activity"] == activity][:400]
#    plot_activity(activity, subset)

# Define column name of the label vector
LABEL = "activity"
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df[LABEL] = le.fit_transform(df["activity"].values.ravel())
df1[LABEL] = le.fit_transform(df1["activity"].values.ravel())

# %%

print("\n--- Reshape the data into segments ---\n")



print("\n--- Fit the model ---\n")

# The EarlyStopping callback monitors training accuracy:
# if it fails to improve for two consecutive epochs,
# training stops early
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='exp.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]



# Hyper-parameters
BATCH_SIZE = 400
EPOCHS = 10


#for i in range(5):

# Differentiate between test set and training set
#df_test = df[df['user-id'] <= i*4]
df_train = df
df_test = df1





# Normalize features for training data set




#df_train['EDA'] = feature_normalize(df['EDA'])


#df_test['HR'] = feature_normalize(df1['HR'])
#df_test['HRV'] = feature_normalize(df1['HRV'])
#df_test['Raw'] = feature_normalize(df1['Raw'])
#df_test['EDA'] = feature_normalize(df1['EDA'])
#df_test['aX'] = feature_normalize(df1['aX'])
#df_test['aY'] = feature_normalize(df1['aY'])
#df_test['aZ'] = feature_normalize(df1['aZ'])
#df_test['aDIR'] = feature_normalize(df1['aDIR'])
#df_test['gX'] = feature_normalize(df1['gX'])
#df_test['gY'] = feature_normalize(df1['gY'])
#df_train['gZ'] = feature_normalize(df1['gZ'])




# Round in order to comply to NSNumber from iOS
#df_train = df_train.round({'x-axis': 6, 'y-axis': 6, 'z-axis': 6})

# Reshape the training data into segments
# so that they can be processed by the network

x_train, y_train = create_segments_and_labels(df_train,TIME_PERIODS,STEP_DISTANCE,LABEL)
#x_test2, y_test2 = create_segments_and_labels(df_test,TIME_PERIODS,STEP_DISTANCE,LABEL)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)


# Accuracy on training data: 99%
# Accuracy on test data: 91%

# %%



print("\n--- Reshape data to be accepted by Keras ---\n")

# Inspect x data
print('x_train shape: ', x_train.shape)
# Displays (20869, 40, 3)
print(x_train.shape[0], 'training samples')
# Displays 20869 train samples

# Inspect y data
print('y_train shape: ', y_train.shape)
# Displays (20869,)

# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
num_classes = le.classes_.size
print(list(le.classes_))

# Set input_shape / reshape for Keras
# Remark: acceleration data is concatenated in one array in order to feed
# it properly into coreml later, the preferred matrix of shape [40,3]
# cannot be read in with the current version of coreml (see also reshape
# layer as the first layer in the keras model)
input_shape = (num_time_periods*num_sensors)
x_train = x_train.reshape(x_train.shape[0], input_shape)
x_test = x_test.reshape(x_test.shape[0], input_shape)

print('x_train shape:', x_train.shape)
# x_train shape: (20869, 120)
print('input_shape:', input_shape)
# input_shape: (120)

# Convert type for Keras otherwise Keras cannot process the data
#x_train = x_train.astype("float32")
#y_train = y_train.astype("float32")

#x_test2 = x_test2.astype("float32")
#y_test2 = y_test2.astype("float32")


# %%

# One-hot encoding of y_train labels (only execute once!)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print('New y_train shape: ', y_train.shape)
# (4173, 6)

# %%
keras.backend.set_learning_phase(0)

# %%

print("\n--- Create neural network model ---\n")

# 1D CNN neural network
model_m = Sequential()
model_m.add(Reshape((TIME_PERIODS, num_sensors,1), input_shape=(input_shape,)))

model_m.add(Conv2D(100, (10,1), activation='relu', input_shape=(TIME_PERIODS, num_sensors, 1))) #added 1 at the end also changed from Conv1D to Conv2D and changed kernal size from 10 to (10,1)
model_m.add(Conv2D(100, (10,1), activation='relu'))

model_m.add(BatchNormalization(axis = -1)) #commented out for android app model
model_m.add(MaxPooling2D(2,1)) #changed from 1d to 2d and added 1 at the end. Also was 3 for all sensors now 2 for just raw HR



model_m.add(Conv2D(160, (10,1), activation='relu'))

model_m.add(Conv2D(160, (10,1), activation='relu'))
model_m.add(BatchNormalization(axis = -1)) #commented out for android app model


model_m.add(GlobalAveragePooling2D())
model_m.add(Dropout(0.5))



#model_m.add(LSTM(ndim=3, return_sequences=True))

model_m.add(BatchNormalization(axis = -1))

model_m.add(Dense(num_classes, activation='softmax'))

print(model_m.summary())

model_m.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])

print("\n--- Learning curve of model training ---\n")


print("\n--- Check against test data ---\n")

# Normalize features for training data set
#df_test['x-axis'] = feature_normalize(df_test['x-axis'])
#df_test['y-axis'] = feature_normalize(df_test['y-axis'])
#df_test['z-axis'] = feature_normalize(df_test['z-axis'])
#df_test['EDA'] = feature_normalize(df_test['EDA'])

#df_test = df_test.round({'x-axis': 6, 'y-axis': 6, 'z-axis': 6})

#x_test, y_test = create_segments_and_labels(df_test,
#                                            TIME_PERIODS,
#                                            STEP_DISTANCE,
#                                            LABEL)



# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)

#x_test = x_test.astype("float32")
#y_test = y_test.astype("float32")

#y_test = np_utils.to_categorical(y_test, num_classes)



In [ ]:
df[LABEL]

In [ ]:
num_classes

In [ ]:
#from sklearn.model_selection import train_test_split
#x_train, x_test3, y_train, y_test3 = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10,  shuffle=True) # Define the split - into 2 folds
kf.get_n_splits(x_train) # returns the number of splitting iterations in the cross-validator
print(kf)
i=0

for train_index, test_index in kf.split(x_train):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, Y_train, Y_test = x_train[train_index], x_train[test_index], y_train[train_index], y_train[test_index]


    history = model_m.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                     # callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)


    score = model_m.evaluate(X_test, Y_test, verbose=1)

    print("\nAccuracy on test data: %0.2f" % score[1])
    print("\nLoss on test data: %0.2f" % score[0])

    # %%

    print("\n--- Confusion matrix for test data ---\n")

    y_pred_test = model_m.predict(X_test)


    if i==0:
        y_pred_test_all = y_pred_test
        Y_test_all = Y_test
        X_test_all = X_test
    if i>0:
        y_pred_test_all = np.concatenate((y_pred_test_all, y_pred_test))
        Y_test_all = np.concatenate((Y_test_all, Y_test))
        X_test_all = np.concatenate((X_test_all, X_test))


    # Take the class with the highest probability from the test predictions
    max_y_pred_test = np.argmax(y_pred_test, axis=1)
    max_y_test = np.argmax(Y_test, axis=1)

    show_confusion_matrix(max_y_test, max_y_pred_test)

    # %%

    print("\n--- Classification report for test data ---\n")

    print(classification_report(max_y_test, max_y_pred_test))

In [ ]:
score = model_m.evaluate(x_test, y_test, verbose=1)

y_pred_test = model_m.predict(x_test)

max_y_pred_test_all = np.argmax(y_pred_test, axis=1)
max_y_test_all = np.argmax(y_test, axis=1)

show_confusion_matrix(max_y_test_all, max_y_pred_test_all)
print("\n--- Classification report for test data ---\n")

print(classification_report(max_y_test, max_y_pred_test, digits=4))
print("\nAccuracy on test data: %0.5f" % score[1])
print("\nLoss on test data: %0.5f" % score[0])

In [ ]:
score = model_m.evaluate(X_test_all, Y_test_all, verbose=1)

max_y_pred_test_all = np.argmax(y_pred_test_all, axis=1)
max_y_test_all = np.argmax(Y_test_all, axis=1)

show_confusion_matrix(max_y_test_all, max_y_pred_test_all)
print("\n--- Classification report for test data ---\n")

print(classification_report(max_y_test, max_y_pred_test))
print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])

In [ ]:
print(model_m.outputs)


In [ ]:
print(model_m.inputs)

In [ ]:
model_m.save('my_model.h5')

In [ ]:
x_test = tf.convert_to_tensor(X_train,dtype=tf.int8)


#def representative_dataset():
#    yield([tf.reshape(x_test[i],(1,1))])
#  for i in range(300):


def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices((x_test)).batch(1).take(100):
    yield [tf.dtypes.cast(data, tf.int8)]


model = tf.keras.models.load_model('my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]





#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.inference_input_type = tf.int8
#converter.inference_output_type = tf.int8
#converter.representative_dataset = representative_dataset





tflite_model = converter.convert()
open("stress_int8.tflite", "wb").write(tflite_model)

In [ ]:
model_m.summary()

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat stress_int8.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

In [ ]:
#x_test = tf.convert_to_tensor(X_train,dtype=tf.float32)


#def representative_dataset():
#    yield([tf.reshape(x_test[i],(1,1))])
#  for i in range(300):


def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices((x_test)).batch(1).take(100):
    yield [tf.dtypes.cast(data, tf.float32)]


model = tf.keras.models.load_model('my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]





converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.representative_dataset = representative_dataset





tflite_model = converter.convert()
open("stress3.tflite", "wb").write(tflite_model)

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat stress3.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

In [ ]:
import tensorflow as tf
import tensorflow.keras as K
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
from tensorflow.keras import backend as K, models
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

# load model from saved chackpoint
model = tf.keras.models.load_model('my_model.h5')

# print model layers and input/outputs
print(model.layers)
for input in model.inputs:
  print(input)
for node in model.outputs:
  print(node)

# Load and transform image


# view output
model.predict([1,1])
# array([[0.6071461]], dtype=float32)

In [ ]:
import numpy as np
# Run the model with TensorFlow to get expected results.
TEST_CASES = 20000

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
count=0
same = 0
different = 0

for i in range(TEST_CASES):
  expected = model_m.predict(x_test[i:i+1])
  interpreter.set_tensor(input_details[0]["index"], x_test[i:i+1, :])
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result,decimal=0)
  if np.array_equal(np.round(expected, decimals=0), np.round(result, decimals=0)):
        print("same")
        same += 1
  else:
        print("different")
        different += 1

  count=count+1
  print(count)